# Event Hub End-to-End Example in Jupyter Notebook

Use this notebook to test the Fake Event Hubs setup that we created. This is  an end-to-end example of sending events to an Event Hub, consuming those events, and storing them in an SQLite database.

## Step 0: Install Required Libraries

In [ ]:
!pip install azure-eventhub

## Step 1: Import Required Libraries

In [ ]:
from azure.eventhub import EventHubProducerClient, EventData, EventHubConsumerClient
import sqlite3

We import the required libraries. `azure.eventhub` is used to send and receive events, and `sqlite3` is used to store the events in a database.

## Step 2: Setup SQLite Database

In [ ]:
# Create an in-memory SQLite database
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# Create a table to store events
cursor.execute('''
    CREATE TABLE IF NOT EXISTS events (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        event_data TEXT
    )
''')

conn.commit()

In this step, we create an in-memory SQLite database and a table to store the events. This table will have two columns: an `id` and the `event_data`.

## Step 3: Send Events to Event Hub

In [ ]:
# Connection string and Event Hub details
connection_str = "Endpoint=sb://localhost;SharedAccessKeyName=RootManageSharedAccessKey;SharedAccessKey=SAS_KEY_VALUE;UseDevelopmentEmulator=true;"
eventhub_name = "eh1"

# Create a producer client to send events
producer = EventHubProducerClient.from_connection_string(conn_str=connection_str, eventhub_name=eventhub_name)

# Create a batch and add events
event_data_batch = producer.create_batch()
event_data_batch.add(EventData('EventOne'))
event_data_batch.add(EventData('EventTwo'))

# Send the batch of events to the Event Hub
producer.send_batch(event_data_batch)

print("Events sent.")

In this step, we create a producer client to connect to the Event Hub. We then create a batch of events and send them to the Event Hub using the producer client.

## Step 4: Consume Events from Event Hub and Store in Database

In [ ]:
# Function to store events in the database
def store_event_in_db(cursor, event_data):
    cursor.execute("INSERT INTO events (event_data) VALUES (?)", (event_data,))
    conn.commit()
    print(f"Stored event: {event_data}")

# Event Hub consumer setup
def consume_events_from_eventhub():
    consumer = EventHubConsumerClient.from_connection_string(
        conn_str=connection_str,
        consumer_group="$default",
        eventhub_name=eventhub_name
    )

    def on_event(partition_context, event):
        event_data = event.body_as_str()
        store_event_in_db(cursor, event_data)
        partition_context.update_checkpoint(event)

    with consumer:
        consumer.receive(
            on_event=on_event,
            starting_position="-1"  # Start from the beginning of the partition
        )

consume_events_from_eventhub()

This step involves creating a consumer that reads events from the Event Hub. Each event is stored in the SQLite database using the `store_event_in_db` function.

## Step 5: Query the Stored Events

In [ ]:
# Query and display stored events
cursor.execute("SELECT * FROM events")
rows = cursor.fetchall()

for row in rows:
    print(row)

Finally, we query the SQLite database to show the events that have been stored.